In [1]:
import torch
from transformers import GPT2Tokenizer, T5ForConditionalGeneration 


tokenizer = GPT2Tokenizer.from_pretrained('ai-forever/FRED-T5-1.7B',eos_token='</s>')
model = T5ForConditionalGeneration.from_pretrained('ai-forever/FRED-T5-1.7B')
device='cuda'
model.to(device)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


T5ForConditionalGeneration(
  (shared): Embedding(50364, 1536)
  (encoder): T5Stack(
    (embed_tokens): Embedding(50364, 1536)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=1536, out_features=1536, bias=False)
              (k): Linear(in_features=1536, out_features=1536, bias=False)
              (v): Linear(in_features=1536, out_features=1536, bias=False)
              (o): Linear(in_features=1536, out_features=1536, bias=False)
              (relative_attention_bias): Embedding(32, 24)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): Linear(in_features=1536, out_features=4096, bias=False)
              (wi_1): Linear(in_features=1536, out_features=4096, bias=False)
       

In [2]:
#Prefix <LM>
lm_text='<LM>Принялся Кутузов рассказывать свою историю как он сюда попал. Началось'
input_ids=torch.tensor([tokenizer.encode(lm_text)]).to(device)
outputs=model.generate(input_ids,eos_token_id=tokenizer.eos_token_id,early_stopping=True)
print(tokenizer.decode(outputs[0][1:]))

/root/anaconda3/envs/purple_env/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:433: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(
/root/anaconda3/envs/purple_env/lib/python3.11/site-packages/transformers/generation/utils.py:1133: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


 с того, что он был в армии, служил в артиллерии.</s>


In [3]:
import clickhouse_connect

client = clickhouse_connect.get_client(host='localhost', username='default', password='')

In [4]:
print(client.command('SHOW COLUMNS FROM cbr_content'))

['texts', 'Array(String)', 'NO', '', '\\N', '\ntokens', 'Array(UInt64)', 'NO', '', '\\N', '\nurl', 'String', 'NO', 'PRI SOR', '\\N', '']


In [82]:
import os
import pandas as pd

# Папка с текстовыми файлами
texts_folder = "data/text_files/"

# Чтение аннотаций из CSV
annotations_df = pd.read_csv("data/text_files_name_link.csv")

# Инициализация словаря для хранения данных
data_dict = {"url": [], "texts": []}
number_of_files = 0
# Чтение текстов из файлов
for filename in os.listdir(texts_folder):
    if filename.endswith(".txt"):
        text_path = os.path.join(texts_folder, filename)
        with open(text_path, "r", encoding="utf-16") as file:
            number_of_files += 1
            text_content = file.read()
            url = annotations_df.loc[annotations_df['answer_filename'] == filename, 'answer_link'].values[0]
#             tokens = text_content.split()  # Простой способ токенизации, замените его по необходимости

            # Если URL уже присутствует в словаре, объединяем тексты
            if url in data_dict["url"]:
                index = data_dict["url"].index(url)
                data_dict["texts"][index] += "\n" + text_content
            else:
                data_dict["url"].append(url)
                data_dict["texts"].append(text_content)

# Создание DataFrame
df = pd.DataFrame(data_dict)

# Установка url в качестве primary key
df.set_index("url", inplace=True)

# Вывод DataFrame
print(df)
print(number_of_files)
print(df.shape)

                                                                                        texts
url                                                                                          
https://cbr.ru/faq/insh/                    В каком порядке осуществляется удовлетворение ...
https://cbr.ru/faq/rcb/                     Что нужно, чтобы получить налоговый вычет за И...
https://cbr.ru/faq/fintekh/                 Зачем нужен личный кабинет РФТ?\n\nЕсли вам не...
https://cbr.ru/faq/information_security/    Что делать, если с вашей банковской карты неза...
https://cbr.ru/faq/dkp/                     Проводит ли Банк России операции с целью укреп...
https://cbr.ru/faq/bank_s/                  Я по ошибке перечислил деньги не тому получате...
https://cbr.ru/faq/pens_k_inv/              Что такое «пенсионные накопления», «пенсионные...
https://cbr.ru/faq/foreign_securities/      Сколько времени займет выкуп иностранных ценны...
https://cbr.ru/faq/microfin/                Чем отличаются м

In [8]:
df.columns

Index(['texts'], dtype='object')

In [9]:
df.index

Index(['https://cbr.ru/faq/insh/', 'https://cbr.ru/faq/rcb/',
       'https://cbr.ru/faq/fintekh/',
       'https://cbr.ru/faq/information_security/', 'https://cbr.ru/faq/dkp/',
       'https://cbr.ru/faq/bank_s/', 'https://cbr.ru/faq/pens_k_inv/',
       'https://cbr.ru/faq/foreign_securities/',
       'https://cbr.ru/faq/microfin/', 'https://cbr.ru/faq/w_fin_sector/',
       'https://cbr.ru/faq/ucbr/', 'https://cbr.ru/faq/credit_h/',
       'https://cbr.ru/faq/pnp/', 'https://cbr.ru/faq/nps/',
       'https://cbr.ru/faq/ndo/', 'https://cbr.ru/faq/client/',
       'https://cbr.ru/faqnew/br/',
       'https://cbr.ru/faq/zhaloby-v-bank-rossii-/'],
      dtype='object', name='url')

In [10]:
print(df.iloc[2][0])

Зачем нужен личный кабинет РФТ?

Если вам необходимо подтвердить факт сделки, в любой момент вы можете зайти на Портал госуслуг, найти по поиску «РФТ» и получить выписку по приобретенным в системе продуктам.

Зачем мне покупать услуги на финансовой платформе?

Финансовая платформа делает доступными предложения банков и других организаций по всей России. Вы сможете приобрести финансовый продукт от организаций, которых нет в вашем регионе. Вам необходимо лишь стать клиентом платформы, и вы сможете в одном месте оформить ипотеку, открыть вклад, приобрести облигации или паи ПИФ, оформить ОСАГО — и все это не выходя из дома.

Почему безопасно пользоваться платформой?

По закону № 211-ФЗ «О совершении сделок с использованием финансовых платформ» все платформы должны соблюдать требования по информационной безопасности, защите ваших данных и обеспечивать бесперебойную работу системы. Деятельность платформ находится под контролем Банка России.

Что Финтех Хаб предлагает образовательным организа

/tmp/ipykernel_85647/3093144479.py:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print(df.iloc[2][0])


In [11]:
def split_text_into_chunks(text, chunk_size, overlap):
    """
    Разбивает текст на чанки по словам с указанным размером и перекрытием.

    :param text: Исходный текст.
    :param chunk_size: Размер чанка в словах.
    :param overlap: Перекрытие между чанками в словах.
    :return: Список чанков текста.
    """
    words = text.split()
    chunks = []

    start = 0
    while start < len(words):
        end = start + chunk_size
        chunk = " ".join(words[start:end])
        chunks.append(chunk)
        start += chunk_size - overlap

    return chunks

 # Исходный текст
input_text = df.iloc[2][0]

# Размер чанка и перекрытие
chunk_size = 20
overlap = 4

# Разбиваем текст на чанки
chunks = split_text_into_chunks(input_text, chunk_size, overlap)

# Выводим результат
for i, chunk in enumerate(chunks):
    print(f"Chunk {i + 1}: {chunk}")

Chunk 1: Зачем нужен личный кабинет РФТ? Если вам необходимо подтвердить факт сделки, в любой момент вы можете зайти на Портал госуслуг,
Chunk 2: зайти на Портал госуслуг, найти по поиску «РФТ» и получить выписку по приобретенным в системе продуктам. Зачем мне покупать услуги
Chunk 3: Зачем мне покупать услуги на финансовой платформе? Финансовая платформа делает доступными предложения банков и других организаций по всей России. Вы
Chunk 4: по всей России. Вы сможете приобрести финансовый продукт от организаций, которых нет в вашем регионе. Вам необходимо лишь стать клиентом
Chunk 5: необходимо лишь стать клиентом платформы, и вы сможете в одном месте оформить ипотеку, открыть вклад, приобрести облигации или паи ПИФ,
Chunk 6: облигации или паи ПИФ, оформить ОСАГО — и все это не выходя из дома. Почему безопасно пользоваться платформой? По закону
Chunk 7: пользоваться платформой? По закону № 211-ФЗ «О совершении сделок с использованием финансовых платформ» все платформы должны соблюдать т

/tmp/ipykernel_85647/887566665.py:23: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  input_text = df.iloc[2][0]


In [12]:
for i, chunk in enumerate(chunks):
    #Prefix <LM>
    lm_text = '<LM>' + chunk
    input_ids = torch.tensor([tokenizer.encode(lm_text)])
    print(f"Chunk {i + 1}: {input_ids}, length: {len(input_ids[0])}")

Chunk 1: tensor([[50357, 32003,  6040, 17154,  9399,  2407,   789,    35,  1178,  1337,
          2003, 25111,  5809, 15154,    16,   282,  3591,  2083,   387,  2890,
         18958,   309, 15376,   300,  1087,   272, 44105,    16]]), length: 28
Chunk 2: tensor([[50357,   431,  1774,   309, 15376,   300,  1087,   272, 44105,    16,
          3089,   334, 42981,   478, 13929,   789,   470,   289,  3604,  8761,
          2977,   334,  7391,  3250,   282,  6829,  2304,  2675,    18,  7132,
           784, 13501,  6196]]), length: 33
Chunk 3: tensor([[50357, 32003,   784, 13501,  6196,   309, 15861, 29205,    35, 36757,
          2232, 30904,  5073,  3772,   758,  8369,  6802,   289,  1888,  7257,
           334,  3025,  1175,    18,  1012]]), length: 25
Chunk 4: tensor([[50357,  1369,  3025,  1175,    18,  1012,  7936, 10850, 31333, 12294,
           360,  7257,    16,  1639,  1003,   282, 11104,  9804,    18,  3379,
          2003,  1472,  1852, 35055]]), length: 24
Chunk 5: tensor([[503

In [140]:
import torch

def pad_or_trim_tokens(tensor, target_length, pad_value=0):
    """
    Выполняет дополнение или обрезку тензора

    :param tensors: Тензор для обработки.
    :param target_length: Целевая длина после обработки.
    :param pad_value: Значение для дополнения (по умолчанию 0).
    :return: Тензор дополненный или обрезанный до target_length.
    """

    current_length = tensor.shape[0]
    result = np.full((1, target_length), pad_value)
    result[0][:current_length] = tensor[:target_length]
    return result 

    
# Размер чанка и перекрытие
chunk_size = 10
overlap = 4

# Размер токена
target_length = 40
padding_token = 0

database = pd.DataFrame(columns=["url", "texts", "tokens"])

for i, row in enumerate(df.iloc):
    chunks = split_text_into_chunks(row["texts"], chunk_size, overlap)
    tokens = []
    for chunk in chunks:
        input_ids = torch.tensor(tokenizer.encode(chunk)).numpy()
        tokens.append(pad_or_trim_tokens(input_ids, target_length, padding_token))
    new_row = {"url": df.index[i], "texts": chunks, "tokens": tokens}
    database = pd.concat([database, pd.DataFrame(new_row)], ignore_index=True)
        
print(database)

                                             url  \
0                       https://cbr.ru/faq/insh/   
1                       https://cbr.ru/faq/insh/   
2                       https://cbr.ru/faq/insh/   
3                       https://cbr.ru/faq/insh/   
4                       https://cbr.ru/faq/insh/   
...                                          ...   
7113  https://cbr.ru/faq/zhaloby-v-bank-rossii-/   
7114  https://cbr.ru/faq/zhaloby-v-bank-rossii-/   
7115  https://cbr.ru/faq/zhaloby-v-bank-rossii-/   
7116  https://cbr.ru/faq/zhaloby-v-bank-rossii-/   
7117  https://cbr.ru/faq/zhaloby-v-bank-rossii-/   

                                                  texts  \
0     В каком порядке осуществляется удовлетворение ...   
1     кредиторов страховой организации после отзыва ...   
2     нее лицензии на осуществление страхования? Пор...   
3     удовлетворения требований кредиторов определяе...   
4     финансового состояния страховой организации. Е...   
...                  

In [141]:
prompt = "Зачем нужен личный кабинет РФТ?"
input_ids = torch.tensor([tokenizer.encode(prompt)])
print(f"Prompt token {input_ids}, length: {len(input_ids[0])}")


Prompt token tensor([[32003,  6040, 17154,  9399,  2407,   789,    35]]), length: 7


In [142]:
print(df.iloc[4]['texts'])

Проводит ли Банк России операции с целью укрепить или ослабить рубль?

Введение режима плавающего валютного курса означает отказ от проведения Банком России регулярных валютных интервенций в целях воздействия на курс рубля. Политика центрального банка при таком курсе состоит в том, чтобы в нормальных условиях не вмешиваться в рыночные процессы и позволить курсу рубля выполнять свою роль встроенного стабилизатора.
Однако Банк России продолжает внимательно следить за ситуацией на валютном рынке и может проводить операции с иностранной валютой (прямые интервенции, валютное репо или валютные свопы) для поддержания финансовой стабильности. При чрезвычайных обстоятельствах также могут вводиться временные ограничения на операции с валютой, которые тоже влияют на курс, но необязательно означают отход от режима плавающего курса.
В качестве угрозы для финансовой стабильности Банк России рассматривает такую динамику обменного курса, которая может привести к формированию устойчивых девальвационных

In [143]:
print(np.array(database['tokens']))

[array([[  677,  6595,  4666, 10074, 30237, 12794, 38502, 32119,  3320,
          1011,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0]])
 array([[  273,  2752,  2632, 32119,  3320,  1011, 30310,   329,  1710,
         22773,   309, 28405,     0,     0,     0,     0,     0,     0,
             0,     0]])
 array([[  968, 22773,   309, 28405, 22852,    35, 36025, 25733, 12794,
         38502, 14903,     0,     0,     0,     0,     0,     0,     0,
             0,     0]])
 ...
 array([[   12, 23419,    13,  7593,    17,  3161,    17,   646,   473,
         12833, 37117,   477, 20306, 15100, 27186,   814, 29161,  6945,
          5482, 24777]])
 array([[19560, 23753,   814, 29161,  6945,  5482, 24777,  1175,  4452,
         38798,    16,   749, 27766,    18,     0,     0,     0,     0,
             0,     0]])
 array([[  273,   570, 21571,  2201,    16,   749, 27766,    18,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
      

In [145]:
import faiss
import numpy as np

def prepare_data(database):
    """
    Подготавливает данные для Faiss.

    :param database: Словарь с данными (url, tokens).
    :return: Нумерические векторы для Faiss.
    """
    urls = list(database.index)
    vectors = np.vstack(database['tokens'].to_numpy())
    
    return urls, vectors

def build_index(vectors):
    """
    Создает и возвращает Faiss индекс.

    :param vectors: Нумерические векторы.
    :return: Faiss индекс.
    """
    print(vectors.shape)
    index = faiss.IndexFlatL2(target_length)
    index.add(vectors)
    return index

def search_index(index, query_vector, k):
    """
    Выполняет поиск вектора по промпту в Faiss индексе.

    :param index: Faiss индекс.
    :param query_vector: Вектор запроса.
    :param k: Количество ближайших соседей для поиска.
    :return: Индексы ближайших соседей и расстояния до них.
    """
    distances, indices = index.search(query_vector.reshape(1, -1), k)
    return indices, distances

if __name__ == "__main__":
    # Подготовка данных
    urls, vectors = prepare_data(database)

    # Создание Faiss индекса
    index = build_index(vectors)

    # Пример поиска по промпту
    prompt = "Проводит ли Банк России операции с целью укрепить или ослабить рубль?"
    query_vector = torch.tensor([tokenizer.encode(prompt)])[0]
    query_vector = pad_or_trim_tokens(query_vector, target_length, padding_token)

    k_neighbors = 5
    result_indices, result_distances = search_index(index, query_vector, k_neighbors)

    # Вывод результатов
    print(f"Ближайшие соседи для {prompt}:")
    for i, index in enumerate(result_indices[0]):
        print(f"Ближайший сосед {i + 1}: {urls[index]}, расстояние: {result_distances[0][i]}")
        
    
    for idx in result_indices[0]:
        print(database.iloc[idx]['texts'], '\n')


(7118, 20)
Ближайшие соседи для Проводит ли Банк России операции с целью укрепить или ослабить рубль?:
Ближайший сосед 1: 5398, расстояние: 428678720.0
Ближайший сосед 2: 6468, расстояние: 540144896.0
Ближайший сосед 3: 2918, расстояние: 555314368.0
Ближайший сосед 4: 3758, расстояние: 574215040.0
Ближайший сосед 5: 1765, расстояние: 623008192.0
подписей в информационной системе. При этом распорядительным актом юридического лица 

лицо может приобрести памятные и инвестиционные монеты за рубежом? Банк 

недоступности указанных в вышеозначенных рекомендациях источников или отсутствия в них 

требуется. Ранее поданное заявление становится недействительным. Для перехода в другой 

не переходите на сайты по ссылкам из подозрительных писем. Подробнее 

